In [12]:
# Basic libraries
import numpy as np
import pandas as pd
import os

# Image processing
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Model building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [13]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected.")

No GPUs detected.


In [14]:
# Path to the CSV file
data_path = "../dataset/fer2013/fer2013/fer2013.csv"  # Update this path to your CSV file

# Load the dataset
df = pd.read_csv(data_path)

# Inspect the first few rows
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [15]:
# Convert pixel values from string to numpy array
def preprocess_images(df):
    images = []
    for pixel_sequence in df["pixels"]:
        img_array = np.array(pixel_sequence.split(), dtype="float32")
        img_array = img_array.reshape(48, 48, 1)  # Reshape to 48x48 and single channel
        img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
    return np.array(images)

# Preprocess images and extract labels
X = preprocess_images(df)
y = df["emotion"].values  # Emotion labels

# Check the shape of images and labels
print("Image data shape:", X.shape)  # Should be (num_samples, 48, 48, 1)
print("Label data shape:", y.shape)  # Should be (num_samples,)

Image data shape: (35887, 48, 48, 1)
Label data shape: (35887,)


In [16]:
# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42, stratify=y_temp)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (28709, 48, 48, 1)
Validation set shape: (5024, 48, 48, 1)
Test set shape: (2154, 48, 48, 1)


In [17]:
# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=15,            # Rotate images by up to 15 degrees
    width_shift_range=0.1,        # Shift images horizontally by up to 10% of the width
    height_shift_range=0.1,       # Shift images vertically by up to 10% of the height
    shear_range=0.1,              # Shear intensity (shear angle in degrees)
    zoom_range=0.1,               # Zoom in/out by up to 10%
    horizontal_flip=True,         # Randomly flip images horizontally
    fill_mode='nearest'           # Fill in new pixels with the nearest value
)

# Fit the generator to the training data
datagen.fit(X_train)

print("Data augmentation setup complete.")


Data augmentation setup complete.


In [18]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(7, activation="softmax")  # 7 output classes for each emotion
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

c:\Users\andri\miniconda3\envs\cv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 619,015 (2.36 MB)

 Trainable params: 619,015 (2.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("model/emotions.keras", save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.2420 - loss: 1.8194 - val_accuracy: 0.3696 - val_loss: 1.6140
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.3673 - loss: 1.6123 - val_accuracy: 0.4536 - val_loss: 1.4423
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.4260 - loss: 1.4841 - val_accuracy: 0.4982 - val_loss: 1.3339
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.4594 - loss: 1.3991 - val_accuracy: 0.5113 - val_loss: 1.2842
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.4860 - loss: 1.3561 - val_accuracy: 0.5197 - val_loss: 1.2453
Epoch 6/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.5008 - loss: 1.3019 - val_accuracy: 0.5287 - val_loss: 1.2308
Epoch 7/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.5096 - loss: 1.2791 - val_accuracy: 0.5366 - val_loss: 1.2100
Epoch 8/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.5193 - loss: 1.2566 - 

In [20]:
# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Generate classification report
y_pred = np.argmax(model.predict(X_val), axis=1)
print(classification_report(y_val, y_pred))

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5987 - loss: 1.0689
Validation Loss: 1.085464358329773
Validation Accuracy: 0.5959395170211792
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
              precision    recall  f1-score   support

           0       0.46      0.52      0.49       694
           1       0.64      0.30      0.41        76
           2       0.52      0.26      0.34       717
           3       0.80      0.83      0.81      1258
           4       0.46      0.50      0.48       851
           5       0.69      0.79      0.74       560
           6       0.52      0.59      0.55       868

    accuracy                           0.60      5024
   macro avg       0.59      0.54      0.55      5024
weighted avg       0.59      0.60      0.59      5024



In [21]:
try:
    model.save("model/mystery.h5")
    print("Model saved successfully.")
except Exception as e:
    print(f"Error saving model: {e}")

Model saved successfully.
